In [1]:
import os
import nibabel as nib
import numpy as np
import cv2
from tqdm import tqdm
from skimage import restoration, io, color
from skimage.measure import blur_effect
import pandas as pd
from utils import get_basename, create_save_nifti
from scipy.ndimage import binary_fill_holes
from scipy.ndimage import zoom
import numpy

In [15]:
fg_scores_path = 'data/EXAMES/Calcium_Scores_Estimations/Fake_Gated/avg=4/130/calcium_score_estimations_avg=4.csv'
gated_scores_path = 'data/EXAMES/Calcium_Scores_Estimations/Gated/All Slices/calcium_score_estimations_dilate_it=5_dilate_k=5.csv'

In [16]:
fake_gated = pd.read_csv(fg_scores_path)
gated = pd.read_csv(gated_scores_path)

gated.rename(columns={'Lesion Gated': 'Gated', 'Pacient': 'patient'}, inplace=True)
fake_gated = pd.merge(fake_gated, gated[['Gated', 'patient']], on='patient', how='left') 

In [17]:
fake_gated.head()

,patient,Escore,Lesion,Heart Mask,Lesion Error,Heart Mask Error,Gated
0,183077,7.0,0,224006,7.0,-223999.0,1
1,74657,725.0,702,241510,23.0,-240785.0,707
2,181613,0.0,33,181441,-33.0,-181441.0,0
3,176253,1082.0,916,164574,166.0,-163492.0,405
4,180545,421.0,129,201016,292.0,-200595.0,139


In [18]:
risk2_erros = fake_gated[(fake_gated.Escore >= 400) & (fake_gated.Lesion < 400)]

In [19]:
print(risk2_erros.shape)

(2, 7)


In [23]:
risk2_erros[['patient', 'Escore', 'Lesion', 'Gated']].head()

,patient,Escore,Lesion,Gated
4,180545,421.0,129,139
46,177233,424.0,379,409


In [22]:
risk2 = fake_gated[fake_gated.Escore >= 400]
print(risk2.shape)

(6, 7)


In [25]:
risk2_gated = gated[(gated.Escore >= 400) & (gated.Gated < 400)]

In [27]:
risk2_gated[['patient', 'Escore', 'Gated']].head()

,patient,Escore,Gated
5,180545,421.0,139
39,182447,405.0,262


In [28]:
print(fake_gated[fake_gated.patient == 182447])

    patient  Escore  Lesion  Heart Mask  Lesion Error  Heart Mask Error  Gated
35   182447   405.0     557      196548        -152.0         -196143.0    262
